In [ ]:
import geopandas as gpd
import pandas as pd
from copy import deepcopy
from geopandas import GeoDataFrame
from shapely.ops import unary_union

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
gdf_center = gpd.read_file('/content/drive/MyDrive/水道管劣化予測/src/国土数値情報/土地利用メッシュ/gdf_center.shp')

In [ ]:
gdf_width = gpd.read_file('/content/drive/MyDrive/水道管劣化予測/src/数値地図/道路/3.shp', encoding='shift-jis') #このデータは道路中心線

In [ ]:
#座標系を変更
src_proj = 4326 # 変換前の座標系を指定
dst_proj = 6678  # 変換後の座標系を指定

# ポイント（ダムデータ、TransformPointの引数は緯度,経度の順番で指定）
gdf_center.crs = f'epsg:{src_proj}'  # 変換前座標を指定
gdf_center = gdf_center.to_crs(epsg=dst_proj)  # 変換後座標に変換

gdf_width.crs = f'epsg:{src_proj}'
gdf_width = gdf_width.to_crs(epsg=dst_proj)

In [ ]:
#高速道路のデータフレームの頭にmainroadをつける

#すべてのカラムを取得
mainroad_columns = gdf_width.columns.to_list()

#ループで各カラムを取得
for i in mainroad_columns[:22]:
  new_column = 'mainroad_' + i
  gdf_width.rename(columns={f'{i}': f'{new_column}'}, inplace=True)

In [ ]:
#空間結合をする
result = gpd.sjoin_nearest(gdf_center, gdf_width, how="left",lsuffix='point',rsuffix='road')
#空間結合された道路のgeometryを追加
result = pd.merge(result, gdf_width[['mainroad_rID', 'geometry']], on='mainroad_rID')
#geometryの名前を変更
result.rename(columns={'geometry_y':'nearest_mainroad'}, inplace=True)
#メモリの解放
del gdf_width

In [ ]:
#dictance_railwyカラムを追加
result['distance_mainroad'] = None

for i in range((len(result) // 50)+1):
    try:
        # ラインデータをマルチラインストリングに結合
        merged_lines = unary_union(list(result['nearest_mainroad'][50*i:50*(i+1)]))

        # ポイントとマルチラインストリングとの最短距離を計算
        result['distance_mainroad'][50*i:50*(i+1)] = result.geometry_x[50*i:50*(i+1)].apply(lambda p: p.distance(merged_lines))
        #一回ごとにメモリを解放
        del merged_lines
    except Exception as e:
        # エラーが発生した場合の処理
        print(f"Error occurred at the last batch: {e}")
        # ラインデータをマルチラインストリングに結合
        merged_lines = unary_union(list(result['nearest_mainroad'][50*(i+1):]))

        result['distance_mainroad'][50*(i+1):] = result.geometry_x[50*(i+1):].apply(lambda p: p.distance(merged_lines))

ストリーミング出力は最後の 5000 行に切り捨てられました。
<ipython-input-9-2062619472e9>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['distance_mainroad'][50*i:50*(i+1)] = result.geometry_x[50*i:50*(i+1)].apply(lambda p: p.distance(merged_lines))
<ipython-input-9-2062619472e9>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['distance_mainroad'][50*i:50*(i+1)] = result.geometry_x[50*i:50*(i+1)].apply(lambda p: p.distance(merged_lines))
<ipython-input-9-2062619472e9>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/p

In [ ]:
#重複を削除
result = result[~result.Meshcode.duplicated()]

#indexを並び替え
result.reset_index(inplace=True)

In [ ]:
result_columns = result.columns.to_list()[6:]
result_columns[:0] = ['Meshcode']
result[result_columns].to_csv('/content/drive/Shareddrives/geomap/周辺環境データ構築/dev/奥州市/奥州市　最近傍主要道路/奥州市_主要道路.csv', encoding='shift-jis', index=False)